In [1]:
#in V34 we are eliminating the aux input. Correcting the training data.

#Version 31 used Quora competition embedding data. The submit button did not work saying exterior 
# data not allowed. Can do internal analysis using gap-development, which is same as test.
# Need to change training data source from gap-dev to gap-training. 

#Version 32 is the first attempted implementation of IsLayer.

import os
import csv
import json
import string
import keras
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
from sklearn.metrics import log_loss
import seaborn as sns
color = sns.color_palette()
from math import floor
import spacy
import zipfile

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics, ensemble, naive_bayes, linear_model
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
import lightgbm as lgb

import time
from tqdm import tqdm
import math
from sklearn.model_selection import train_test_split

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, GRU, CuDNNGRU, Conv1D
from keras.layers import Bidirectional, GlobalMaxPool1D, Concatenate, Add, Flatten, CuDNNLSTM
from keras.models import Model
from keras import backend as K
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.engine.topology import Layer
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np # linear algebra

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

from wordcloud import WordCloud, STOPWORDS
from collections import defaultdict

# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

Using TensorFlow backend.


ModuleNotFoundError: No module named 'spacy'

In [2]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(u'A few days later, Abigail complained that Elizabeth was pinching her and tearing at her bowels')

for token in doc:
    print(token.text, token.pos_, token.tag_, token.dep_)

A DET DT det
few ADJ JJ amod
days NOUN NNS npadvmod
later ADV RB advmod
, PUNCT , punct
Abigail PROPN NNP nsubj
complained VERB VBD ROOT
that ADP IN mark
Elizabeth PROPN NNP nsubj
was VERB VBD aux
pinching VERB VBG ccomp
her PRON PRP dobj
and CCONJ CC cc
tearing VERB VBG conj
at ADP IN prep
her ADJ PRP$ poss
bowels NOUN NNS pobj


In [3]:
def word_locate(sentence, location): 
    count_words = 0
    count_chars = 2 #2 is to count for the two spaces in the beginning
    for word in sentence.split():
        count_words += 1
        if location == count_chars:
            return word, count_words
        count_chars += len(word)
        count_chars += 1 #for space

In [4]:
def curr_prev_sentence(sentence, loc):
    current_sentence = ""
    prev_sentence = ""
    detect = 0
    count = 0
    for char in sentence:
        count += 1
        current_sentence += char
        if char == "." and detect == 0:
            prev_sentence = current_sentence 
            current_sentence = ""
        if char == "." and detect == 1:
            return current_sentence, prev_sentence
        if count == loc:
            detect = 1

In [5]:
def find_subject(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == "nsubj":
            return token.text
    return "none"

In [6]:
text1 = "After a few years of almost no work -- although he was a guest star on Lou Grant and Charlie's Angels in the late 1970s, he once summed up the 1970s as 'I cried and did a lot of gardening' -- he was hired in 1979 for his best-known role, self-made millionaire Palmer Cortlandt on ABC's long-running soap opera All My Children. Initially hired for only one year, he remained on contract through 2009. For much of his first decade on the show, Palmer was a ruthless villain, totally possessive of his daughter, Nina and violently threatening his ex-wife Daisy with being attacked by dobermans when she came back from the dead."

In [7]:
text2 = "When onlookers expressed doubt, claiming that the Proctor family was well regarded in the community, the girl promptly came out of her trance and told them it was all for 'sport'. On March 29, 1692, Abigail Williams and Mercy Lewis again said they were being tormented by Elizabeth's spectre. A few days later, Abigail complained that Elizabeth was pinching her and tearing at her bowels, and said she saw Elizabeth's spectre as well as John's."

In [8]:
current, prev = curr_prev_sentence(text2, 360)
print(current)

 A few days later, Abigail complained that Elizabeth was pinching her and tearing at her bowels, and said she saw Elizabeth's spectre as well as John's.


In [9]:
candidate = find_subject(current)
print(candidate)

Abigail


In [10]:
word, loc = word_locate(text2, 360) 
print(word, " ", loc)

her   60


In [11]:
with open('../input/gap-other-training-data/gap-training.tsv') as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    train_X = []
    train_y = []
    tr_label_A_list = []
    tr_label_B_list = []
    for row in reader:
        text = row['Text']
        text = text.lower()
        new_textA = text
        labelA = 0
        labelB = 0
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(text)):
            if char_idx == int(row['A-offset']):
                new_textA = new_textA[:char_idx+check] + "person_loc " + new_textA[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textA = new_textA[:char_idx+check] + "pronom_loc " + new_textA[char_idx+check:]
                check = 11
        if row['A-coref'] == 'TRUE':
            labelA = 1
        train_X.append(new_textA)
        train_y.append(labelA)
        new_textB = text
        label = 0
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(text)):
            if char_idx == int(row['B-offset']):
                new_textB = new_textB[:char_idx+check] + "person_loc " + new_textB[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textB = new_textB[:char_idx+check] + "pronom_loc " + new_textB[char_idx+check:]
                check = 11
        if row['B-coref'] == 'TRUE':
            labelB = 1

        train_X.append(new_textB)
        train_y.append(labelB)
        
        tr_label_A_list.append(labelA)
        tr_label_B_list.append(labelB)

In [12]:
with open('../input/gap-other-training-data/gap-training.tsv') as tsvfile:
    reader = csv.DictReader(tsvfile, dialect='excel-tab')
    testA_X = []
    testA_y = []
    testB_X = []
    testB_y = []
    label_A_list = []
    label_B_list = []
    for row in reader:
        text = row['Text']
        text = text.lower()
        new_textA = text
        labelA = 0
        labelB = 0
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(text)):
            if char_idx == int(row['A-offset']):
                new_textA = new_textA[:char_idx+check] + "person_loc " + new_textA[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textA = new_textA[:char_idx+check] + "pronom_loc " + new_textA[char_idx+check:]
                check = 11
        if row['A-coref'] == 'TRUE':
            labelA = 1
        testA_X.append(new_textA)
        testA_y.append(labelA)
        new_textB = text
        label = 0
        check = 0 #to allign the text in correct location after first insert
        for char_idx in range(len(text)):
            if char_idx == int(row['B-offset']):
                new_textB = new_textB[:char_idx+check] + "person_loc " + new_textB[char_idx+check:]
                check = 11
            if char_idx == int(row['Pronoun-offset']):
                new_textB = new_textB[:char_idx+check] + "pronom_loc " + new_textB[char_idx+check:]
                check = 11
        if row['B-coref'] == 'TRUE':
            labelB = 1

        testB_X.append(new_textB)
        testB_y.append(labelB)
        
        label_A_list.append(labelA)
        label_B_list.append(labelB)

In [13]:
print(len(train_X))

4000


In [14]:
maxlen = 120
embed_size = 300
max_features = 5000

In [15]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer_list = list(train_X)
tokenizer.fit_on_texts(tokenizer_list)

train_X = tokenizer.texts_to_sequences(train_X)
testA_X = tokenizer.texts_to_sequences(testA_X)
testB_X = tokenizer.texts_to_sequences(testB_X)

train_X = pad_sequences(train_X, maxlen=maxlen)
testA_X = pad_sequences(testA_X, maxlen=maxlen)
testB_X = pad_sequences(testB_X, maxlen=maxlen)

In [16]:
word_index = tokenizer.word_index
max_features = len(word_index)

In [17]:
!wget http://nlp.stanford.edu/data/glove.840B.300d.zip

--2019-03-24 06:49:43--  http://nlp.stanford.edu/data/glove.840B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... failed: Temporary failure in name resolution.
wget: unable to resolve host address ‘nlp.stanford.edu’


In [18]:
with zipfile.ZipFile("glove.840B.300d.zip","r") as zip_ref:
    zip_ref.extractall()
!ls 'cased_glove_300'

FileNotFoundError: [Errno 2] No such file or directory: 'glove.840B.300d.zip'

In [19]:
def load_glove(word_index):
    EMBEDDING_FILE = '../input/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if word == "pronom_loc":
            embedding_vector = np.ones(300)*0.85 # the number is arbitrary, just some vector
        if word == "person_loc":
            embedding_vector = np.ones(300)*0.25 # the number is arbitrary, just some vector
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        
    return embedding_matrix 

In [20]:
def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

In [21]:
def dupli_row(x, num_copies, axis):
    y = x
    for i in range(num_copies-1):
        y = np.concatenate((y,x), axis= axis)
    return y

In [22]:
class IsLayer(Layer):
    #Layer to be used after a dense one. It will multiply all the elements with each other.
    #In a sense, it allows the neurons to have a say on  each others' outputs. This layer, hopefully,
    #compares the relative importance of neurons. A dominant neuron can have more say via its weight.
    #The idea follows from attention layer, but is more basic than that. As it is multiplicative, it is 
    #an alternative to the vanilla additive layer where outputs are added at the next layer.
    
    def __init__(self, **kwargs):
        super(IsLayer, self).__init__(**kwargs)
    
    def build(self, input_shape):
        #Create a trainable weight variable for this layer.
        self.W = self.add_weight(name='W', 
                                 shape=(input_shape[1], 1), 
                                 initializer='uniform',
                                 trainable=True)
        super(IsLayer, self).build(input_shape)
        
    def call(self, x):
        x_W = K.dot(x, self.W)
        x_new = x*x_W 
        return x_new
    
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

In [23]:
class AttentionWithContext(Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self,
                 W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):


        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)


    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight((input_shape[-1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= K.cast(mask, K.floatx())


        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [24]:
train_y = np.asarray(train_y)

In [25]:
embedding_matrix = load_glove(word_index)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning:

arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.



In [26]:
inp1 = Input(shape=(maxlen,))

model1_out = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp1)
model1_out = Bidirectional(CuDNNGRU(256, return_sequences=True))(model1_out)
model1_out = AttentionWithContext()(model1_out)
model1_out = Dropout(0.1)(model1_out)

merged_out = Dense(32, activation="relu")(model1_out)
merged_out = Dropout(0.1)(merged_out)
merged_out = Dense(1, activation="sigmoid")(merged_out)
model = Model(inputs=inp1, outputs=merged_out)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 120)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 120, 300)          6783900   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 120, 512)          857088    
_________________________________________________________________
attention_with_context_1 (At (None, 512)               263168    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)      

In [27]:
model.fit(train_X, train_y, batch_size=512, epochs=20, validation_data=(train_X, train_y))

Instructions for updating:
Use tf.cast instead.
Train on 4000 samples, validate on 4000 samples
Epoch 1/20
4000/4000 [==============================] - 3s 842us/step - loss: 0.6901 - acc: 0.5445 - val_loss: 0.6867 - val_acc: 0.5567
Epoch 2/20
4000/4000 [==============================] - 1s 246us/step - loss: 0.6887 - acc: 0.5465 - val_loss: 0.6861 - val_acc: 0.5567
Epoch 3/20
4000/4000 [==============================] - 1s 245us/step - loss: 0.6734 - acc: 0.5843 - val_loss: 0.6471 - val_acc: 0.6455
Epoch 4/20
4000/4000 [==============================] - 1s 246us/step - loss: 0.6455 - acc: 0.6367 - val_loss: 0.6181 - val_acc: 0.6672
Epoch 5/20
4000/4000 [==============================] - 1s 247us/step - loss: 0.6231 - acc: 0.6665 - val_loss: 0.5996 - val_acc: 0.6883
Epoch 6/20
4000/4000 [==============================] - 1s 248us/step - loss: 0.6052 - acc: 0.6865 - val_loss: 0.5825 - val_acc: 0.7068
Epoch 7/20
4000/4000 [==============================] - 1s 247us/step - loss: 0.5940 - a

In [28]:
pred_valA_y = model.predict(testA_X, batch_size=512, verbose=1)
pred_valB_y = model.predict(testB_X, batch_size=512, verbose=1)

2000/2000 [==============================] - 0s 67us/step


In [29]:
print(len(pred_valB_y), " ", len(pred_valA_y))

2000   2000


In [30]:
print("Test score A:", log_loss(pred_valA_y,testA_y))

ValueError: Unknown label type: (array([[0.01749572],
       [0.96493477],
       [0.15171367],
       ...,
       [0.94990075],
       [0.8518851 ],
       [0.25072706]], dtype=float32),)

In [31]:
print("Test score B:", log_loss(pred_valB_y,testB_y))

ValueError: Unknown label type: (array([[0.26928896],
       [0.02994871],
       [0.14222583],
       ...,
       [0.01984984],
       [0.00883377],
       [0.06046918]], dtype=float32),)

In [32]:
out_df = pd.DataFrame({"ID":test_ids})

NameError: name 'test_ids' is not defined

In [33]:
out_df['A'] = [max(float(val),0.33) for val in list(pred_valA_y)]
out_df['B'] = [max(float(val),0.33) for val in list(pred_valB_y)]

NameError: name 'out_df' is not defined

In [34]:
out_df['NEITHER'] = [max((1-float(val)),0.33) for val in list(pred_valA_y)]

NameError: name 'out_df' is not defined

In [35]:
out_df.to_csv("submission.csv", index=False)

NameError: name 'out_df' is not defined